In [3]:
import telegram
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import logging
import pandas as pd
import pandahouse
import os
import datetime as dt

connection = {
    'host': 'https://clickhouse.lab.karpov.courses',
    'password': 'dpo_python_2020',
    'user': 'student',
    'database': 'simulator_20220220'
}

In [5]:
chat_id = 1295860693
bot = telegram.Bot(token='5291500178:AAFQqAVPSG4ad90QdyE-IaZH-Ai8ow2oxFs')
#bot = telegram.Bot(token=os.environ.get("report_bot_token"))
# для удобства построения графиков в запрос можно добавить колонки date, hm
q =  ''' select 
                        Full_time, date, Hour_min,
                        sum(activ_feed_users) as activ_feed_users,
                        sum(likes) as likes,
                        sum(views) as views,
                        sum(activ_mess_users) as activ_mess_users,
                        sum(messages) as messages

                    from
                        (select 
                            all_unique_users.Full_time as Full_time ,
                            all_unique_users.date as date,
                            all_unique_users.Hour_min as Hour_min,
                            feed_table.activ_feed_users as activ_feed_users,
                            feed_table.likes as likes,
                            feed_table.views as views,
                            likes*100/views as CTR,
                            mess_table.activ_mess_users AS activ_mess_users,
                            mess_table.messages as messages
                        from
                            (Select 
                                toStartOfFifteenMinutes(time) as Full_time,
                                toDate(Full_time) as date,
                                formatDateTime(Full_time, '%R') as Hour_min,
                                user_id
                            from simulator_20220220.feed_actions
                            group by Full_time, date, Hour_min, user_id
                            union all
                            Select 
                                toStartOfFifteenMinutes(time) as Full_time,
                                toDate(Full_time) as date,
                                formatDateTime(Full_time, '%R') as Hour_min,
                                user_id
                            from simulator_20220220.message_actions
                            group  by Full_time, date, Hour_min, user_id) as all_unique_users
                        left join
                            (select
                                toStartOfFifteenMinutes(time) as Full_time,
                                toDate(Full_time) as date,
                                formatDateTime(Full_time, '%R') as Hour_min,
                                user_id,
                                count(distinct user_id) as activ_feed_users,
                                countIf(user_id, action = 'like') as likes,
                                countIf(user_id, action = 'view') as views
                            from simulator_20220220.feed_actions
                            group by Full_time, date, Hour_min, user_id) as feed_table 
                        on all_unique_users.user_id = feed_table.user_id and all_unique_users.Full_time = feed_table.Full_time
                        left join
                            (select
                                toStartOfFifteenMinutes(time) as Full_time,
                                toDate(Full_time) as date,
                                formatDateTime(Full_time, '%R') as Hour_min,
                                user_id,
                                count(distinct user_id) as activ_mess_users,
                                count(user_id) as messages
                            from simulator_20220220.message_actions
                            group by Full_time, date, Hour_min, user_id) as mess_table
                        on all_unique_users.user_id = mess_table.user_id and all_unique_users.Full_time = mess_table.Full_time
                        where Full_time >=  today() - 1 and Full_time < toStartOfFifteenMinutes(now()) ) as virtual_table
                    group by Full_time, date, Hour_min
                    order by Full_time '''
df_allert = pandahouse.read_clickhouse(q, connection=connection)

In [9]:
df_allert['CTR'] = round(df_allert.likes*100/df_allert.views,2)

In [12]:
for metric in df_allert.iloc[:, 3:].columns:
    

,Full_time,date,Hour_min,activ_feed_users,likes,views,activ_mess_users,messages,CTR
0,2022-03-29 00:00:00,2022-03-29,00:00,391,1544,6811,81,91,22.67
1,2022-03-29 00:15:00,2022-03-29,00:15,402,1569,6989,73,93,22.45
2,2022-03-29 00:30:00,2022-03-29,00:30,387,1426,6523,58,67,21.86
3,2022-03-29 00:45:00,2022-03-29,00:45,403,1576,7147,67,79,22.05
4,2022-03-29 01:00:00,2022-03-29,01:00,366,1437,6514,65,69,22.06
...,...,...,...,...,...,...,...,...,...
141,2022-03-30 11:15:00,2022-03-30,11:15,605,2187,9757,149,196,22.41
142,2022-03-30 11:30:00,2022-03-30,11:30,593,2181,9606,156,202,22.70
143,2022-03-30 11:45:00,2022-03-30,11:45,583,2076,9227,136,175,22.50
144,2022-03-30 12:00:00,2022-03-30,12:00,641,2230,9961,125,163,22.39
